In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 23.0MB/s 
     |████████████████████████████████| 890kB 48.2MB/s 
     |████████████████████████████████| 2.9MB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4b6997b7597fb5a2c6369a6b816ca89258e6a69b6d06acdc6f6fc8cfc5257230
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.7.0+cu101


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Bert_BadGay

/content/drive/My Drive/Colab Notebooks/Bert_BadGay


In [ ]:
!ls

AML_NER.json		      thirdLayer_tainingdata.csv
BadGuyPredicter.ipynb	      thirdLayer_tainingdatav2.csv
FirstLayer_IOT.ipynb	      thirdLayer_tainingdatav3.csv
firstLayer_tainingdata.csv    thirdLayer_tainingdatav4.csv
firstLayer_tainingdataV2.csv  ThridLayer.ipynb
firstLayer_tainingdataV3.csv  titletrainData.csv
Layer3SubmitV2.csv	      trained_layer3_model
Layer3TrainingDataV2.csv      trained_model
NewForBadGuy_tainingdata.csv  金融犯罪預測模型.gslides


# **資料整理與清洗**
- 針對.JSON調整成CSV使用

In [ ]:
import json
with open('AML_NER.json', 'r', encoding='utf-8') as f:
  output = json.load(f)

In [ ]:
print(output[0])
print(output[1])
print(output[2])

{'pid': 1, 'link': 'https://news.cnyes.com/news/id/4352432', 'title': '0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35facebook commentFONT SIZEICON PRINT78981DD6-B6FA-435C-B969-0D6ABD01805C0「時間序列與量化交易研討會」邀集專家學者進行量化交易、智能投資等相關議題研討。(圖:鄭心芸攝)相關個股元大期6023近年來投資市場波動越來越明顯 ### 省略內文 ### 都有非常巨大的助益。', 'context': '0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35facebook commentFONT SIZEICON PRINT78981DD6-B6FA-435C-B969-0D6ABD01805C0「時間序列與量化交易研討會」邀集專家學者進行量化交易、智能投資等相關議題研討。(圖:鄭心芸攝)相關個股元大期6023近年來投資市場波動越來越明顯 ，追求低波動、絕對報酬的量化交易備受注目。專家表示，採用量化交易策略投資台股，不管是處於多頭或是空頭市場，績效及波動度均可領跑大盤，甚至比國內投資台股的股票型基金及ETF的波動率還低，表現也更為穩定。大數據時代來臨，風行歐美50年的量化交易儼然成為顯學，台灣亦開始重視此一趨勢發展，也因此，中華機率統計學會及台北科技大學管理學院攜手主辦，並由元大期貨、摩根亞太量化交易等公司擔任協辦單位，今(7/5)日舉辦「時間序列與量化交易研討會」，就目前熱門的量化交易、智能投資等相關議題進行研討。越來越多的基金公司重視量化交易，全球規模較大的避險基金多採行量化交易，包括橋水基金(BridgewaterAssociates)、AQR資產管理公司、曼氏集團(ManGroup)、文藝復興科技(RenaissanceTechnologies)等全球知名避險基金。摩根亞太集團董事長張堯勇指出，避險基金規模約為5兆美元，採取量化交易的基金規模約1兆美元，比重佔了20%，代表量化交易的操作績效好，才會有那麼高的比重。量化交易的操作績效不亞於價值投資及技術投資，被譽為數學天才、最賺錢的基金經理人，文藝復興對沖基金創始人詹姆斯·西蒙斯(

In [ ]:
# Init output datarow
output_title = []
output_context = []
output_label = []

In [ ]:
# 整理訓練資料 & 超過512只擷取500 (預留Buff)
for oneData in output :
  bert_ids = tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(oneData['context'])))
  # - Collect Content
  if len(bert_ids) < 512 :
    output_context.append(oneData['context'])
  else :
    output_context.append(oneData['context'][:500])
  # - Collect Label
  if oneData['name'] == "[]" :
    output_label.append(0)
  else :
    output_label.append(1)
   

In [ ]:
print(len(output_context))
print(len(output_label))

5023
5023


In [ ]:
# Write CSV test
dataframeWriter = pd.DataFrame({'label':output_label,'context':output_context })
# 將DataFrame儲存為csv,index表示是否顯示行名，default=True
dataframeWriter.to_csv("/content/drive/My Drive/Colab Notebooks/Bert_BadGay/NewForBadGuy_tainingdata.csv",index=True,sep=',')

# **第一層模型訓練任務**
- Bert　: 訓練「內文」與「是否是金融犯罪者新聞」

In [ ]:
trainPd = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Bert_BadGay/NewForBadGuy_tainingdata.csv', sep=',', header=0)
trainPd.head(5)

,Unnamed: 0,label,context
0,0,0,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...
1,1,0,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ，北市府勞動局認定業者未依職...
2,2,0,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...
3,3,0,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...
4,4,0,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...


In [ ]:
max_seq_len = -1
tindex = -1
for index,row in trainPd.iterrows():
  bert_ids = tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(row['context'])))
  if(len(bert_ids)>max_seq_len):
    max_seq_len = len(bert_ids)
    tindex = index
    
print("最長問句長度For Testing:",max_seq_len , tindex)

最長問句長度For Testing: 511 417


In [ ]:
max_seq_len = 511

In [ ]:
print(len(trainPd))

5023


In [ ]:
print(trainPd.loc[417]['context'])

2019.11.26 21:58【梁靜茹被劈很大6】梁靜茹抵押房產幫老公出資餐廳收了還欠款文｜娛樂組攝影｜攝影組全文朗讀00:00 / 00:00今年8月本刊爆出梁靜茹、趙元同分居 ，開啟了兩人的婚變風暴，而且兩人共同扶養兒子。（翻攝自趙元同微博）相較之下，梁靜茹可說是慘愛而且人財兩失，媒體曾經如此報導：「她過去對於Tony（趙元同英文名字）的事業非常支持，幾年前曾經拿名下房產抵押，出資給Tony開餐廳，後來餐廳收了，他倆離婚後，Tony仍有欠款。」更誇張的是，聽說趙元同還會在外抱怨，梁靜茹因為工作無法陪他、照顧小孩等等，等於是一手拿人家錢，把錢賠光，一手又對女生出手大方，還要貧嘴。11/2214：36，趙元同開車門準備去林以婕家，臨上車做了一個很像吐口香糖渣的奇怪舉動。梁靜茹的老公之前曾經開漢堡店，她還跟好友范瑋琪去捧場並且上傳照片宣傳。（翻攝自范瑋琪臉書）難怪婚變對於梁靜茹打擊會如此大，除了由親友輪流陪伴，大部分都是媽媽陪她度過低潮。被問是否前夫有小三才導致離婚？梁靜茹眼眶含淚沒做回答，等於默認。友人覺得梁靜茹受很大的委屈，近半年不但終日以淚洗面，至今仍得接受心理諮商，也難怪頻頻去找陳綺貞的前男友鍾成虎談心， 因而爆出過從甚密。更新時間｜2019.11.26 12:38


In [ ]:
tmp = trainPd[(trainPd['label'] == 1)]
print(len(tmp))
tmp.head(5)

372


,Unnamed: 0,label,context
14,14,1,台北地院審理生產東亞燈具的上市公司中國電器掏空案 ### 省略內文 ### 掏空公司資產2億...
17,17,1,自稱房產幽默大師的王派宏 ，涉吸金捲款25億落跑！他自稱炒房專家，在全台授課分享理財，卻遭指...
37,37,1,12年前爆發的花蓮縣壽豐鄉鯉魚潭風景區的「綠湖國際大飯店」開發弊案 內政部前都計委員李威儀確...
40,40,1,〔記者楊國文／台北報導〕「台灣第一家」有限公司 ，被查出將砷含量超標的工業用碳酸鎂摻入胡椒粉...
65,65,1,苗栗縣一群外籍配偶向本報投訴 ，希望朱女出面，給眾人一個交代。（記者鄭名翔攝）2019-09...


In [ ]:
tmp2 = trainPd[(trainPd['label'] == 0)][:372]
print(len(tmp2))
tmp2.head(5)

372


,Unnamed: 0,label,context
0,0,0,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...
1,1,0,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ，北市府勞動局認定業者未依職...
2,2,0,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...
3,3,0,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...
4,4,0,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...


In [ ]:
v2DataSet = tmp
v2DataSet = v2DataSet.append(tmp2)

In [ ]:
print(len(v2DataSet))
v2DataSet = v2DataSet.sample(frac=1)
v2DataSet.head()

744


,Unnamed: 0,label,context
235,235,0,0理財基金【保德信投信】市場不確定性高多元資產提升防禦力保德信投信※來源：保德信投信2019...
327,327,0,台股台股盤勢【群益投顧】指數開低後即在盤下來回震盪 ，收盤仍跌落在12000點之下，後市整理...
386,386,0,中央銀行去年曾警示無現金社會存在四大風險。圖／聯合報系資料照片台灣因金融環境方便 ，無現金社...
355,355,0,在美國川普指責中國操縱匯率之後 ，匯豐銀行股價下挫近3%，凸顯其在應對地緣政治緊張所面臨的風...
1765,1765,1,證期雙分析師陳韋霖應自2014年開始利用證券公司貴賓室開課 ，推銷「比例式價差」、「選擇權雙...


In [ ]:
mTrainingSet = trainPd.iloc[:3500]
mTestingSet = trainPd.iloc[3501:]

In [ ]:
mTrainingSet = v2DataSet.iloc[:520]
mTestingSet = v2DataSet.iloc[520:]

In [ ]:
def fillZero( tokens , max_len ) :  
  while len(tokens)<max_len:
    tokens.append(0)
  return tokens

In [ ]:
def ToBertFormat( title , label ) :
  # From makeDataset : TensorDataset(all_input_ids, all_input_segment_ids, all_input_masks, all_answer_lables)
  selLen = -1 
  # [0] : id
  tokens = tokenizer.tokenize( title )
  content_id = tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokens))
  selLen = len(content_id)
  content_id = fillZero(content_id, max_seq_len)
  # [1] : segment ids
  segment_id = [0]*max_seq_len
  # [2] : mask
  mask_id = [1]*selLen + [0]*(max_seq_len-selLen)
  # [3] : answer label
  label_id = label

  return content_id , segment_id , mask_id , label_id

In [ ]:
tmp_content_id , tmp_segment_id , tmp_mask_id , tmp_label_id  = ToBertFormat(trainPd.loc[0]['context'],trainPd.loc[0]['label'])
print(tmp_content_id)
print(tmp_segment_id)
print(tmp_mask_id)
print(tmp_label_id)
print(len(tmp_content_id))
print(len(tmp_segment_id))
print(len(tmp_mask_id))


[101, 121, 4415, 6512, 1825, 7032, 7030, 1265, 769, 3211, 6841, 3724, 5179, 2205, 1841, 6992, 3300, 3126, 2205, 2834, 4281, 4220, 2356, 7056, 773, 5206, 6250, 5442, 6972, 2552, 5712, 9160, 120, 8155, 120, 8137, 8130, 131, 8198, 12917, 100, 100, 100, 118, 100, 118, 100, 118, 100, 118, 100, 519, 3229, 7279, 2415, 1154, 5645, 7030, 1265, 769, 3211, 4777, 6245, 3298, 520, 6913, 7415, 2201, 2157, 2119, 5442, 6868, 6121, 7030, 1265, 769, 3211, 510, 3255, 5543, 2832, 6536, 5023, 4685, 7302, 6359, 7539, 4777, 6245, 511, 113, 1756, 131, 6972, 2552, 5712, 3109, 114, 4685, 7302, 943, 5500, 1039, 1920, 3309, 8183, 8748, 6818, 2399, 889, 2832, 6536, 2356, 1842, 3797, 1240, 6632, 889, 6632, 3209, 7549, 8024, 6841, 3724, 856, 3797, 1240, 510, 5179, 2205, 1841, 6992, 4638, 7030, 1265, 769, 3211, 991, 1358, 3800, 4680, 511, 2201, 2157, 6134, 4850, 8024, 2967, 4500, 7030, 1265, 769, 3211, 5032, 4526, 2832, 6536, 1378, 5500, 8024, 679, 5052, 3221, 5993, 3176, 1914, 7531, 2772, 3221, 4958, 7531, 2356, 184

In [ ]:
# 出 Embedinig
data_content_ids = [] 
data_segment_ids =[]
data_mask_ids = []
data_label_ids = []
for index,row in mTrainingSet.iterrows(): 
  tmp_content_id , tmp_segment_id , tmp_mask_id , tmp_label_id  = ToBertFormat(row['context'],row['label'])
  data_content_ids.append(tmp_content_id) 
  data_segment_ids.append(tmp_segment_id) 
  data_mask_ids.append(tmp_mask_id)
  data_label_ids.append(tmp_label_id)
print( len(data_content_ids) )

520


In [ ]:
# 出 Embedinig
test_data_content_ids = [] 
test_data_segment_ids =[]
test_data_mask_ids = []
test_data_label_ids = []
for index,row in mTestingSet.iterrows(): 
  tmp_content_id , tmp_segment_id , tmp_mask_id , tmp_label_id  = ToBertFormat(row['context'],row['label'])
  test_data_content_ids.append(tmp_content_id) 
  test_data_segment_ids.append(tmp_segment_id) 
  test_data_mask_ids.append(tmp_mask_id)
  test_data_label_ids.append(tmp_label_id)
print( len(test_data_content_ids) )

224


In [ ]:
print( len(test_data_label_ids) )

224


In [ ]:
traingData_features = { 'input_ids':data_content_ids,'input_segment_ids':data_segment_ids,'input_masks':data_mask_ids,'answer_lables':data_label_ids }
testingData_features = { 'input_ids':test_data_content_ids,'input_segment_ids':test_data_segment_ids,'input_masks':test_data_mask_ids,'answer_lables':test_data_label_ids }

In [ ]:
def makeDataset(data_feature):
    input_ids = data_feature['input_ids']
    input_segment_ids = data_feature['input_segment_ids']
    input_masks = data_feature['input_masks']
    answer_lables = data_feature['answer_lables']

    all_input_ids = torch.tensor([input_id for input_id in input_ids], dtype=torch.long)
    all_input_segment_ids = torch.tensor([input_segment_id for input_segment_id in input_segment_ids], dtype=torch.long)
    all_input_masks = torch.tensor([input_mask for input_mask in input_masks], dtype=torch.long)
    all_answer_lables = torch.tensor([answer_lable for answer_lable in answer_lables], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_input_segment_ids, all_input_masks, all_answer_lables)
    return dataset

In [ ]:
import os
import pickle
import torch
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
trainingDataSet = makeDataset(traingData_features)
testingDataSet = makeDataset(testingData_features)

In [ ]:
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [ ]:
bert_config, bert_class, bert_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)

# Set use gpu
device = torch.device("cuda")

train_dataloader = DataLoader(trainingDataSet ,batch_size=2 ,shuffle=True)
test_dataloader = DataLoader(testingDataSet ,batch_size=2 ,shuffle=True)

config = bert_config.from_pretrained('bert-base-chinese',num_labels = 2)
model = bert_class.from_pretrained('bert-base-chinese', from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
model.to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
Learning_rate = 5e-6       # 學習率
optimizer = AdamW(optimizer_grouped_parameters, lr=Learning_rate, eps=1e-8)

for epoch in range(5):
    # 訓練模式
    model.train()
    All_train_correct = 0.0
    AllTrainLoss = 0.0
    count = 0
    for batch_dict in train_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)
        
        # From makeDataset : TensorDataset(all_input_ids, all_input_segment_ids, all_input_masks, all_answer_lables)
        # [0] : id
        # [1] : segment id
        # [2] : mask
        # [3] : answer label

        outputs = model(
            batch_dict[0],
            token_type_ids = batch_dict[1],
            attention_mask = batch_dict[2],
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]
        
        train_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_train_correct += train_correct
        AllTrainLoss += loss.item()
        count += 1

        model.zero_grad()
        loss.backward()
        optimizer.step()
        
    Average_train_correct = round(All_train_correct/count, 3)
    Average_train_loss = round(AllTrainLoss/count, 3)

    # 測試模式
    model.eval()
    All_test_correct = 0.0
    AllTestLoss = 0.0
    count = 0
    for batch_dict in test_dataloader:
        batch_dict = tuple(t.to(device) for t in batch_dict)

        outputs = model(
            batch_dict[0],
            token_type_ids = batch_dict[1],
            attention_mask = batch_dict[2],
            labels = batch_dict[3]
            )
        loss, logits = outputs[:2]

        test_correct = compute_accuracy(logits, batch_dict[3])       # 計算正確率
        All_test_correct += test_correct
        AllTestLoss += loss.item()

        count += 1
    
    Average_test_correct = round(All_test_correct/count, 3)
    Average_test_loss = round(AllTestLoss/count, 3)

    print('第' + str(epoch+1) + '次' + '訓練模式，loss為:' + str(Average_train_loss) + ' 正確率為' + str(Average_train_correct)+ '，測試模式，loss為:' + str(Average_test_loss) + ' 正確率為' + str(Average_test_correct))

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained('trained_iot_v2_model')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

第1次訓練模式，loss為:0.315 正確率為88.654，測試模式，loss為:0.239 正確率為89.286
第2次訓練模式，loss為:0.104 正確率為95.769，測試模式，loss為:0.071 正確率為98.214
第3次訓練模式，loss為:0.031 正確率為99.423，測試模式，loss為:0.128 正確率為95.982
第4次訓練模式，loss為:0.007 正確率為100.0，測試模式，loss為:0.073 正確率為97.321
第5次訓練模式，loss為:0.005 正確率為100.0，測試模式，loss為:0.067 正確率為97.321
